<a href="https://colab.research.google.com/github/helenatong/oc_p4_Anticipez_les_besoins_en_consommation_de_batiments/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**

---

## **Prérequis**
- Télécharger [dataset nettoyé](https://drive.google.com/file/d/1-EuzI1ibLAG_1brg_v3MKcDtEgKDFIdl/view?usp=drive_link) dans Google Drive
- Modifier le chemin du fichier dans le code


---

---

## **Sommaire**

### **I. Installation et mise en place de l'environnement de travail**

### **II. Définitions des fonctions auxiliaires**

### **III. Prédiction de la variable TotalGHGEmissions**
**A. Entrainement du premier modèle: régression linéaire**

# **I. Installation et mise en place de l'environnement de travail**

In [184]:
# importation des librairies
import numpy as np
import pandas as pd

import zipfile
import os

import re

import sys

import missingno as msno
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression

import datetime

from google.colab import drive

In [185]:
# informations sur l'environnement virtuel
print('Version des librairies :')
print('Python        : ' + sys.version)
print('NumPy         : ' + np.version.full_version)
print('Pandas        : ' + pd.__version__)
print('Matplotlib    : ' + mpl.__version__)
print('Seaborn       : ' + sns.__version__)

now  = datetime.datetime.now().isoformat()
print('Run date      : ' + now)

Version des librairies :
Python        : 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
NumPy         : 1.25.2
Pandas        : 2.0.3
Matplotlib    : 3.7.1
Seaborn       : 0.13.1
Run date      : 2024-05-11T09:43:23.591398


In [186]:
# configurations de l'affichages des données
pd.set_option('display.max_columns', None)

In [187]:
# connexion au drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [188]:
# vérification de l'importation
df = pd.read_parquet('/content/drive/MyDrive/openclassrooms/project_list/project_4/data/cleaned_2016_Building_Energy_Benchmarking.pq')
df.head(2)

,OSEBuildingID,BuildingType,PrimaryPropertyType,Neighborhood,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,ENERGYSTARScore,SiteEnergyUse(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu),TotalGHGEmissions,BuildingAge,LastYearENERGYSTARCertified,NbYearENERGYSTARCertified
0,1,NonResidential,Hotel,DOWNTOWN,1.0,12,88434,0,88434,Hotel,88434.0,missing,0.0,missing,0.0,60.0,7226362.5,2003882.0,3946027.0,1276453.0,249.98,89,0.0,0
1,2,NonResidential,Hotel,DOWNTOWN,1.0,11,103566,15064,88502,Hotel,83880.0,Parking,15064.0,Restaurant,4622.0,61.0,8387933.0,0.0,3242851.0,5145082.0,295.86,20,0.0,0


In [189]:
# vérification des données
df.isna().sum(axis=0)

OSEBuildingID                      0
BuildingType                       0
PrimaryPropertyType                0
Neighborhood                       0
NumberofBuildings                  0
NumberofFloors                     0
PropertyGFATotal                   0
PropertyGFAParking                 0
PropertyGFABuilding(s)             0
LargestPropertyUseType             0
LargestPropertyUseTypeGFA          0
SecondLargestPropertyUseType       0
SecondLargestPropertyUseTypeGFA    0
ThirdLargestPropertyUseType        0
ThirdLargestPropertyUseTypeGFA     0
ENERGYSTARScore                    0
SiteEnergyUse(kBtu)                0
SteamUse(kBtu)                     0
Electricity(kBtu)                  0
NaturalGas(kBtu)                   0
TotalGHGEmissions                  0
BuildingAge                        0
LastYearENERGYSTARCertified        0
NbYearENERGYSTARCertified          0
dtype: int64

# **II. Définitions des fonctions et des variables globales**


In [190]:
# TARGET =
ID = 'OSEBuildingID'

In [191]:
# préparation des données pour ML, variable a de l'impact ou pas?
 # normalisation des données
 # encodage
 # dataset de training

 # TARGET 1 - prédire une valeur numérique
#tester simple la régression lineaire

#NEW NOTEBOOK
# TARGET 1 - prédire une valeur numérique
#tester simple la régression lineaire

# entrainement modèle, pipeline, grid search
# choix de la méthode de scoring
# comparer les 4 modèles : ElasticNet, SVM, GradientBoosting, RandomForest
# si iverfitting : tester une méthode ensembliste (bagging, boosting)

# check impact de STAR SCORING avec et sans
#c'est pour comparer ce qui est comparable. ENtre BTP aux services similaires (école <> industrie)

#NEW NOTEBOOK
# TARGET 2 - prédire une valeur numérique
# entrainement modèle

#shap
#ElasticNet (lineaire, ), SVM, GradientBoosting, RandomForest).

#pour savoir si une va est importante ?
# simple : utiliser la lib black box
# compliqué : LASSO (connaissance de l'algo!), cas particulier de l'eslatic net, 0 sur tout

# **III. TARGET 1**

In [192]:
TARGET = 'TotalGHGEmissions'
NUM_FEATURES = ['OSEBuildingID', 'NumberofBuildings', 'NumberofFloors',
       'PropertyGFATotal', 'PropertyGFAParking', 'PropertyGFABuilding(s)',
       'LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseTypeGFA',
       'ThirdLargestPropertyUseTypeGFA', 'ENERGYSTARScore', 'SiteEnergyUse(kBtu)',
       'SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)',
       'BuildingAge', 'LastYearENERGYSTARCertified',
       'NbYearENERGYSTARCertified']
CAT_FEATURES = [f for f in df.columns if f not in NUM_FEATURES and f != TARGET]

df1 = df.copy()

**Trouver les variables prédictifs à l'aide du lasso ? To do si le temps**

#**A. Entrainement d'un modèle linéaire**

In [193]:
# importation des librairies
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, PowerTransformer

In [194]:
# séparation des variables explicatives de la variable à prédire
df1 = df.copy()
y = df1.pop(TARGET)
X = df1

In [195]:
# features engineering
y = np.log(y + 1)

preprocessor = ColumnTransformer(
    transformers=[
        ('encoder_nominal', OneHotEncoder(handle_unknown='ignore',sparse_output=False), CAT_FEATURES),
         ('norma', PowerTransformer(), NUM_FEATURES)
    ],
    remainder='drop'
  )

In [196]:
# création de la pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
  ])

In [205]:
# entrainement des modèles selon plusieurs paramètres
# séparation du dataset (entrainement et test)

param_grid = {
    'regressor__fit_intercept' : [False, True],
    'regressor__positive'      : [False, True]
}

gs = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, verbose=1, n_jobs=-1, refit=True, scoring='r2')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
gs.fit(X_train, y_train)
best_model = gs.best_estimator_

print(f'Meilleurs hyperparamètres: {gs.best_params_}')
print(f'Moyenne des scores: {gs.best_score_.round(3)}')
print('----------------')

# model scoring
print(f'Score sur le train : {best_model.score(X_train, y_train).round(3)}')
print(f'Score sur le test : {best_model.score(X_test, y_test).round(3)}')

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Meilleurs hyperparamètres: {'regressor__fit_intercept': False, 'regressor__positive': True}
Moyenne des scores: 0.867
----------------
Score sur le train : 0.957
Score sur le test : 0.9


In [199]:
best_model.predict(X_test)

array([ 4.02872424,  5.6750542 ,  3.76370015,  2.96738156,  4.54214607,
       -0.33799903,  5.94088061,  4.64926181,  2.95971235,  5.01937414,
        3.28984867,  1.88134273,  3.90053406,  3.67908556,  4.18383384,
        4.14642584,  4.65269081,  3.8941419 ,  6.37032595,  4.92747146,
        3.44816754,  3.55933637,  5.64754271,  6.49421866,  4.43321136,
        4.1979726 ,  2.91603081,  3.20576592,  2.19083346,  2.07746885,
        3.06676767,  4.1387835 ,  3.1916181 ,  4.92025183,  1.86094675,
        4.04087796,  0.91552109,  2.85514451,  7.24376964,  5.27961548,
        6.49349366,  3.98518654,  3.44660687,  3.08430109,  3.84422381,
        6.03699828,  1.19540269,  3.8897076 ,  4.65495365,  5.35542979,
        5.56697862,  4.70281462,  5.60841249,  3.09555317,  2.3320916 ,
        2.12745259,  3.00684673,  3.43244866,  3.87129227,  4.02360932,
        5.56768377,  2.41416986,  2.336074  ,  2.96016857,  4.33180131,
        3.7022273 ,  4.38577556,  4.20733347,  3.71213662,  3.57

##**Entrainement du modèle LASSO**


In [115]:
# On choisie toutes les variables numériques et 3 variables catégorielles
nb_null_value = df.isna().sum(axis=0)
col_to_keep = nb_null_value[nb_null_value < 100].index.to_list()
num_column = df.dtypes[(df.dtypes == 'float64') | (df.dtypes == 'int64')].index.to_list()
cat_to_keep = ['PrimaryPropertyType', 'Neighborhood']
num_to_keep = list(set(col_to_keep).intersection(num_column))
FINAL_COL =  num_to_keep + cat_to_keep #set pour le non itérable, #list pour concat ensuite
FINAL_COL = ['TotalGHGEmissions', 'Electricity(kBtu)', 'SiteEnergyUse(kBtu)', 'NaturalGas(kBtu)'
             , 'NumberofBuildings', 'PrimaryPropertyType', 'Neighborhood']
print((FINAL_COL))

['TotalGHGEmissions', 'Electricity(kBtu)', 'SiteEnergyUse(kBtu)', 'NaturalGas(kBtu)', 'NumberofBuildings', 'PrimaryPropertyType', 'Neighborhood']


In [116]:
TARGET = 'TotalGHGEmissions'
FEATURE = [col for col in FINAL_COL if col != TARGET]
df_train = df[FINAL_COL].dropna()
print(FEATURE)

['Electricity(kBtu)', 'SiteEnergyUse(kBtu)', 'NaturalGas(kBtu)', 'NumberofBuildings', 'PrimaryPropertyType', 'Neighborhood']


In [117]:
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.preprocessing import OneHotEncoder, PowerTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline

In [118]:
# Features engineering
preprocessor = ColumnTransformer(
    transformers=[
        ('encoder_nominal', OneHotEncoder(handle_unknown='ignore',sparse_output=False), cat_to_keep),
         ('norma', PowerTransformer(), ['Electricity(kBtu)', 'SiteEnergyUse(kBtu)', 'NaturalGas(kBtu)', 'NumberofBuildings'])
    ],
    remainder='drop'
  )

In [119]:
pp = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', Lasso()),
])

In [120]:
# On va séparer les variables explicatives de la variable cible.
y = df_train.pop(TARGET)
X = df_train
print(y.shape, X.shape)
# # Question : encodage sur tout le jeu de données ou une partie seulement ? a priori separer puis encoder car pbm de type
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

# X_train_df = pd.DataFrame(data=X_train, columns=FEATURE)
# y_train_df = pd.DataFrame(data=y_train, columns=[TARGET])

# model = PowerTransformer()
# y_train_normed = model.fit_transform(y_train_df)
# X_df = pd.DataFrame(data=X, columns=FEATURE)
# y_df = pd.DataFrame(data=y, columns=[TARGET])
pp.fit(X, y)

(1476,) (1476, 6)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('encoder_nominal',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['PrimaryPropertyType',
                                                   'Neighborhood']),
                                                 ('norma', PowerTransformer(),
                                                  ['Electricity(kBtu)',
                                                   'SiteEnergyUse(kBtu)',
                                                   'NaturalGas(kBtu)',
                                                   'NumberofBuildings'])])),
                ('regressor', Lasso())])

In [125]:
# fit model by using grid search
param_grid = {
    'regressor__alpha'          : [0.0001, 0.001, 0.01, 0.10, 1],
    'regressor__fit_intercept'  : [True, False]
}
grid_search = GridSearchCV(pp, param_grid, cv=5, verbose=1, n_jobs=-1, refit=True, scoring='r2')
grid_search.fit(X, y)
print(grid_search.best_params_)
print('----------------')

# model scoring
best_model = grid_search.best_estimator_
print(f'Model\'s score : {best_model.score(X, y).round(3)}') #est R2 ? Bon va falloir revoir pipeline etc.

Fitting 5 folds for each of 10 candidates, totalling 50 fits
      Electricity(kBtu)  SiteEnergyUse(kBtu)  NaturalGas(kBtu)  \
0          3.946027e+06         7.226362e+06      1.276453e+06   
1          3.242851e+06         8.387933e+06      5.145082e+06   
2          4.952666e+07         7.258702e+07      1.493800e+06   
3          2.768924e+06         6.794584e+06      1.811213e+06   
4          5.368607e+06         1.417261e+07      8.803998e+06   
...                 ...                  ...               ...   
1471       6.323620e+05         9.320821e+05      2.997200e+05   
1472       3.965461e+05         9.502762e+05      5.537300e+05   
1473       1.792159e+06         5.765898e+06      3.973739e+06   
1474       3.488702e+05         7.194712e+05      3.706010e+05   
1475       4.325542e+05         1.152896e+06      7.203420e+05   

      NumberofBuildings PrimaryPropertyType           Neighborhood  
0                   1.0               Hotel               DOWNTOWN  
1       

In [122]:
lasso_regressor = best_model.named_steps['regressor']

# Access the coefficients of the Lasso
lasso_coefficients = lasso_regressor.coef_

print("Lasso Coefficients:")
print(lasso_coefficients)

# Print number of total variables and total variables with non null parameter
(
    lasso_coefficients.shape,
    lasso_coefficients[lasso_coefficients != 0.].shape
)
# Le modele a simplifié notre equation: il a choisi seulement 14 variables parmis les 17 !

Lasso Coefficients:
[-0.00000000e+00  3.30171588e+03  1.12407265e+02 -0.00000000e+00
  3.02766217e+02 -1.75753305e+02 -0.00000000e+00 -8.10549978e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -1.58726204e+01
  0.00000000e+00  4.54366431e+01 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  8.83098501e+00  0.00000000e+00 -1.56855911e+01
  0.00000000e+00  0.00000000e+00  2.31865198e+01  1.91846001e+02
 -0.00000000e+00 -9.92940285e+00  1.13859383e+01  0.00000000e+00
 -1.05050342e-01 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
  2.61266021e+02  3.00461929e+00  6.11935096e+01  5.73262635e+01]


((36,), (18,))